# Use R Shiny app to create SIR model with `ibm-watsonx-ai`

This notebook contains steps and code to demonstrate support of external R Shiny application code with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository. 

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goals

The learning goals of this notebook are:

-  Persist a R Shiny app in in Watson Machine Learning asset repository.
-  Deploy application for online scoring using client library.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Application upload](#upload)
3. [Web service creation](#deploy)
4. [Clean up](#cleanup)
5. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.1"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.1"
)

```

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [2]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Application upload

In this section you will learn how to store downloaded application
as a WML asset.

#### Download R Shiny app from git project using wget
**Hint**: To install `wget` execute `!pip install wget`.

In [ ]:
import os
import wget

filename = "app.R.zip"
url = "https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd5.1/models/r_shiny/app.R.zip"
if not os.path.isfile(filename):
    filename = wget.download(url)

#### Upload application as data asset

In [7]:
shiny_details = client.shiny.store(
    meta_props={
        client.shiny.ConfigurationMetaNames.NAME: "R Shiny app - SIR",
    },
    file_path=filename
)
shiny_asset_id = client.shiny.get_id(shiny_details)

Creating Shiny asset...
SUCCESS


**Note:** You can see that application is saved in Watson Machine Learning Service.

In [ ]:
client.shiny.list(limit=5)

<a id="deploy"></a>
## 3. Create online deployment
You can use commands bellow to create online deployment for stored application (web service).

In [10]:
deployment = client.deployments.create(
    artifact_id=shiny_asset_id,
    meta_props={
                client.deployments.ConfigurationMetaNames.NAME: "Deployment R Shiny's SIR",
                client.deployments.ConfigurationMetaNames.R_SHINY: {"authentication" : "anyone_with_url" },
                client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name":"S", "num_nodes":1},
    }
)
deployment_id = client.deployments.get_id(deployment)



######################################################################################

Synchronous deployment creation for id: '5b747bc6-d1cb-41fe-900b-aa6650427344' started

######################################################################################


initializing
Note: rshiny_url is deprecated and will be removed in a future release. Use serving_urls instead.
.........
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='53d4f5c0-502a-441c-8341-676d394206f0'
-----------------------------------------------------------------------------------------------




#### Get deployments details

In [ ]:
deployment_id = client.deployments.get_id(deployment)
client.deployments.get_details(deployment_id)

<a id="cleanup"></a>
## 4. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 5. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.